<a href="https://colab.research.google.com/github/motazsaad/infant-cry-care/blob/main/CryCare2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
title = 'CryCareSheet'
spreedSheet = gc.open(title)
audioContSheet = spreedSheet.sheet1

In [ ]:

# reportSheet = spreedSheet.worksheet('report')
# spreedSheet.add_worksheet(title ='report', rows = 200, cols = 100)

In [ ]:
!pip install ffmpeg
!pip install git+https://github.com/jiaaro/pydub.git@master

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=9cf1efe352f435ae65e1a8a189a07c685fbf71653ea3f624649e9b370633fb72
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
  Cloning https://github.com/jiaaro/pydub.git (to revision master) to /tmp/pip-req-build-tbdsr85w
  Running command git clone -q https://github.com/jiaaro/pydub.git /tmp/pip-req-build-tbdsr85w
  Created wheel for pydub: filename=pydub-0.25.1-py2.py3-none-any.whl size=32352 sha256=40bd17b55b90ff1bbd93a7ede6d8d26b3e310627c1401c4080595da05c3e6b27
  Stored in directory: /tmp/pip-ephem-wheel-cache-qk577xq6/wheels/a7/e5/a1/626333b98a5e043f69b7bdd3a989b87b8ccf9005f4b66a6aef
Successfully built pydub


In [ ]:
import glob
import subprocess
import os
from pydub import AudioSegment
from pydub.utils import make_chunks
from pydub.silence import split_on_silence
import pandas as pd
import librosa 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from gspread_dataframe import set_with_dataframe


mainPath = '/content/drive/MyDrive/CryCareDataset/'
originDataPath = mainPath + 'Dataset/'
oneSecDataPath = mainPath + 'oneSecDataset/'
onSilencePath = mainPath + 'onSilence/'
donateACryPath = mainPath + 'donateacry/'
directories = ['Hungry', 'lower gas', 'burp up', 'Sleepy', 'Uncomfortable'] 

In [ ]:
def filterAudioFile ():
  for directory in os.listdir(donateACryPath):
    for file in glob.glob(donateACryPath + directory + '/**.wav'):
      age = file.split('-')[-2]
      filename = file.split('/')[-1]
      newPath = originDataPath + directory + '/' + filename
      if(age == '04' or age == '48' or age == '26'):
        subprocess.call(['ffmpeg', '-i', file,
                      newPath])
filterAudioFile()

In [ ]:
def convertMp3ToWav(path, directories) :
  for directory in directories:
    for filename in glob.glob(path + directory + '/**.mp3'):
        filewithoutext = filename.split('.')
        wavfile = filewithoutext[0] + '.wav'
        subprocess.call(['ffmpeg', '-i', filename,
                      wavfile])
      
#convert all directories to wav
convertMp3ToWav(originDataPath, directories)

In [ ]:
def makeDirectory(newPath, directories):
  try:
    os.mkdir(newPath)
    for directoryName in directories:
      os.mkdir(newPath + directoryName)
  except:
    print('File exists')

#make a new directories in drive for oneSecDataset
makeDirectory(oneSecDataPath, directories)
makeDirectory(onSilencePath, directories)

In [ ]:
#one sec chunks

#Export all of the individual chunks as wav files
def exportChunks(path, directory, filename, chunks):
  # for i, chunk in enumerate(myaudio[::1000]):
  for i, chunk in enumerate(chunks):
    filewithoutext = filename.split('/')
    name = filewithoutext[-1].split('.')[0]
    chunk_name = name + "chunk{0}.wav".format(i)
    # if(len(chunk) > 900): 
    chunk.export(path + directory + '/' + chunk_name, format="wav")

def makeChunks(path, directories) :
  for directoryName in directories:
    for filename in glob.glob(path + directoryName + '/**.wav'):
      myaudio = AudioSegment.from_file(filename , "wav") 
      chunk_length_ms = 1000 # pydub calculates in millisec
      chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
      exportChunks(oneSecDataPath, directoryName, filename, chunks)

makeChunks(originDataPath, directories)


In [ ]:
def makeChunksOnSilence(path, directories):
  for directoryName in directories:
    for filename in glob.glob(path + directoryName + '/**.wav'):
      myaudio = AudioSegment.from_file(filename , "wav") 
      myaudio = myaudio.set_frame_rate(11025)
      # print(len(myaudio), filename)
      chunks = split_on_silence(myaudio, silence_thresh=-40, min_silence_len=500)
      exportChunks(onSilencePath, directoryName, filename, chunks)

makeChunksOnSilence(originDataPath, directories)

In [ ]:
#Store all audio files in dictionary where key: filename, value: label
def createRawAudio(path) :
  raw_audio = dict()
  for directory in directories :
    for filename in glob.glob(path + directory + '/**.wav'):
      raw_audio[filename] = directory
  return raw_audio

In [ ]:
def audioFeatureExtraction(raw_audio, featureExtraction):
  X = pd.DataFrame(columns = np.arange(45), dtype = 'float32').astype(np.float32)
  for i, filename in enumerate(raw_audio):
      if filename.endswith(".wav"):
          audiofile, sr = librosa.load(filename, sr = 11025)
          fingerprint = featureExtraction(y=audiofile, sr=sr)
          x = pd.DataFrame(fingerprint, dtype = 'float32')
          x[44] = raw_audio[filename]
          X.loc[i] = x.loc[0]
  X = X.fillna(0)
  return X

In [ ]:
#Do something with missing values.
print(X)
print(X.isnull().sum())
X = X.fillna(0)

In [ ]:
def trainTestSplit(X):
  y = X[44]
  del X[44]
  X = X.astype(np.float32)
  X_train, X_test, y_train, y_test = train_test_split(X, y)
  return X_train, X_test, y_train, y_test

In [ ]:
def get_scores(classifier, X_train, X_test, y_train, y_test, **kwargs):
        model = classifier(**kwargs)
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        return classification_report(y_test, y_predict, target_names=directories, output_dict=True)
# randomForest =  get_scores(RandomForestClassifier, X_train, X_test, y_train, y_test)
# SVM =  get_scores(SVC, X_train, X_test, y_train, y_test)


In [ ]:
def exportClassificationReport(report, sheetName):
  df = pd.DataFrame(report).transpose()
  df[''] = directories + ['accuracy', 'macro avg', 'weighted avg']
  try:
    spreedSheet.add_worksheet(title =sheetName, rows = 200, cols = 100)
  except:
    print('file exist')
  reportSheet = spreedSheet.worksheet(sheetName)
  set_with_dataframe(reportSheet, df) 
  # return df

In [ ]:
df = pd.DataFrame()
df[''] = directories
def countDirectories(path, dataName):
  audio_count = []
  for directoryName in directories:
    count = 0
    for filename in glob.glob(path + directoryName + '/**.wav'):
      count +=1
    audio_count.append(count)
    count = 0
  df[dataName] = audio_count

countDirectories(originDataPath, 'originData')
countDirectories(onSilencePath, 'onSilence')
countDirectories(oneSecDataPath, 'oneSec')
#save on sheet
set_with_dataframe(worksheet1, df) 

In [ ]:
def makeModel(path, featureExtraction, classifier):
  raw_audio = createRawAudio(path)
  X = audioFeatureExtraction(raw_audio, featureExtraction)
  X_train, X_test, y_train, y_test = trainTestSplit(X)
  model = get_scores(classifier, X_train, X_test, y_train, y_test)
  return (model)

In [ ]:
model = makeModel(originDataPath, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model, 'originData,spectralCentroid,RandomForestClassifier')


In [ ]:
model2 = makeModel(originDataPath, librosa.feature.mfcc, SVC)
exportClassificationReport(model2, 'originData,mfcc,SVC')

In [ ]:
model3 = makeModel(oneSecDataPath, librosa.feature.mfcc, SVC)
exportClassificationReport(model3, 'oneSecData,mfcc,SVC')

In [ ]:
model4 = makeModel(oneSecDataPath, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model4, 'oneSecData,spectralCentroid,RandomForestClassifier')

In [ ]:
model5 = makeModel(onSilencePath, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model5, 'onSilence,spectralCentroid,RandomForestClassifier')

In [ ]:
model6 = makeModel(onSilencePath, librosa.feature.mfcc, RandomForestClassifier)
exportClassificationReport(model6, 'onSilence,mfcc,RF')

In [ ]:
model7 = makeModel(onSilencePath, librosa.feature.mfcc, SVC)
exportClassificationReport(model7, 'onSilence,mfcc,svc')

In [ ]:
model8 = makeModel(onSilencePath, librosa.feature.spectral_centroid, RandomForestClassifier)
exportClassificationReport(model8, 'onSilence,spectralCentroid,RandomForestClassifier')

In [ ]:
model9 = makeModel(onSilencePath, librosa.feature.chroma_stft, RandomForestClassifier)
exportClassificationReport(model9, 'onSilence,chroma_stft,RandomForestClassifier')

In [ ]:
model10 = makeModel(onSilencePath, librosa.feature.chroma_stft, SVC)
exportClassificationReport(model10, 'onSilence,chroma_stft,SVC')